Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128

graph = tf.Graph()

with graph.as_default():

    layer_1_units = 128
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    #Inputs are of length 784 (img pixels), but outputs need to be of size 128 (number hidden units))
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size,  layer_1_units]))
    biases1 = tf.Variable(tf.zeros([layer_1_units]))
    
    #Inputs are of length 128 (number hidden units of layer1), but outputs need to be of size 10 (number of labels))
    weights2 = tf.Variable(
        tf.truncated_normal([layer_1_units, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    def layer(inputs):
        logits1 = tf.matmul(inputs, weights1) + biases1
        layer1_out = tf.nn.relu(logits1)
        logits_out = tf.matmul(layer1_out, weights2) + biases2
        return logits_out
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(layer(tf_train_dataset) , tf_train_labels))
    

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Add the regularization term to the loss.
    loss += 5e-4 * regularizers
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(layer(tf_train_dataset))
    valid_prediction = tf.nn.softmax(layer(tf_valid_dataset))
    test_prediction = tf.nn.softmax(layer(tf_test_dataset))

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 132.766
Minibatch accuracy: 15.6%
Validation accuracy: 29.2%
Minibatch loss at step 500 : 16.4164
Minibatch accuracy: 76.6%
Validation accuracy: 75.9%
Minibatch loss at step 1000 : 13.4955
Minibatch accuracy: 72.7%
Validation accuracy: 79.1%
Minibatch loss at step 1500 : 9.99706
Minibatch accuracy: 77.3%
Validation accuracy: 79.8%
Minibatch loss at step 2000 : 7.89253
Minibatch accuracy: 76.6%
Validation accuracy: 81.6%
Minibatch loss at step 2500 : 6.26154
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 3000 : 4.64872
Minibatch accuracy: 88.3%
Validation accuracy: 83.8%
Test accuracy: 90.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10 == 0):
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 122.408
Minibatch accuracy: 8.6%
Validation accuracy: 29.9%
Test accuracy: 44.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128

graph = tf.Graph()

with graph.as_default():

    layer_1_units = 128
    keep_prob = 0.5
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    #Inputs are of length 784 (img pixels), but outputs need to be of size 128 (number hidden units))
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size,  layer_1_units]))
    biases1 = tf.Variable(tf.zeros([layer_1_units]))
    
    #Inputs are of length 128 (number hidden units of layer1), but outputs need to be of size 10 (number of labels))
    weights2 = tf.Variable(
        tf.truncated_normal([layer_1_units, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    def layer(inputs, dropout=False):
        logits1 = tf.matmul(inputs, weights1) + biases1
        layer1_out = tf.nn.relu(logits1)
        if dropout:
            layer1_out = tf.nn.dropout(layer1_out, keep_prob)
        logits_out = tf.matmul(layer1_out, weights2) + biases2
        return logits_out
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(layer(tf_train_dataset, True) , tf_train_labels))
    

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Add the regularization term to the loss.
    loss += 5e-4 * regularizers
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(layer(tf_train_dataset))
    valid_prediction = tf.nn.softmax(layer(tf_valid_dataset))
    test_prediction = tf.nn.softmax(layer(tf_test_dataset))

In [9]:
num_steps = 3

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10 == 0):
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 149.277
Minibatch accuracy: 14.8%
Validation accuracy: 24.5%
Test accuracy: 55.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [19]:
batch_size = 128

graph = tf.Graph()

initial_learning_rate = 0.3

with graph.as_default():

    layer_1_units = 128
    layer_2_units = 128
    
    keep_prob = tf.placeholder(tf.float32)
    keep_prob = 0.5
    
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    #Inputs are of length 784 (img pixels), but outputs need to be of size 128 (number hidden units))
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size,  layer_1_units]))
    biases1 = tf.Variable(tf.zeros([layer_1_units]))
    
    #Inputs are of length 128 (number hidden units of layer1), but outputs need to be of size 10 (number of labels))
    weights2 = tf.Variable(
        tf.truncated_normal([layer_1_units, layer_2_units]))
    biases2 = tf.Variable(tf.zeros([layer_2_units]))
    
    #Inputs are of length 128 (number hidden units of layer1), but outputs need to be of size 10 (number of labels))
    weights3 = tf.Variable(
        tf.truncated_normal([layer_2_units, num_labels]))
    biases3 = tf.Variable(tf.zeros([num_labels]))
    
    keep_prob
  
    # Training computation.
    def layer(inputs):
        logits1 = tf.matmul(inputs, weights1) + biases1
        layer1_out = tf.nn.relu(logits1)
        
        logits2 = tf.matmul(layer1_out, weights2) + biases2
        layer2_out = tf.nn.dropout(tf.nn.relu(logits2), keep_prob)
        
        logits3 = tf.matmul(layer2_out, weights3) + biases3
        layer3_out = tf.nn.relu(logits3)
        
        #if dropout:
        #    layer2_out = tf.nn.dropout(layer2_out, keep_prob)
        #layers_out = tf.matmul(layer2_out, weights3) + biases3
        return layer3_out
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(layer(tf_train_dataset) , tf_train_labels))
    

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2) +
                  tf.nn.l2_loss(weights3) + tf.nn.l2_loss(biases3))
    
    # Add the regularization term to the loss.
    loss += 5e-4 * regularizers
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(initial_learning_rate).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(layer(tf_train_dataset, ))
    valid_prediction = tf.nn.softmax(layer(tf_valid_dataset))
    test_prediction = tf.nn.softmax(layer(tf_test_dataset))

In [ ]:
num_steps = 500000

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
    tf.initialize_all_variables().run()
    print "Initialized"
    for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 944.503
Minibatch accuracy: 10.2%
Validation accuracy: 11.9%
Minibatch loss at step 1000 : 20.2153
Minibatch accuracy: 9.4%
Validation accuracy: 10.3%
Minibatch loss at step 2000 : 15.573
Minibatch accuracy: 7.0%
Validation accuracy: 10.3%
Minibatch loss at step 3000 : 12.1333
Minibatch accuracy: 8.6%
Validation accuracy: 10.3%
Minibatch loss at step 4000 : 9.58522
Minibatch accuracy: 10.9%
Validation accuracy: 10.3%
Minibatch loss at step 5000 : 7.69757
Minibatch accuracy: 14.1%
Validation accuracy: 10.3%
Minibatch loss at step 6000 : 6.2992
Minibatch accuracy: 11.7%
Validation accuracy: 10.3%
Minibatch loss at step 7000 : 5.26329
Minibatch accuracy: 4.7%
Validation accuracy: 10.3%
Minibatch loss at step 8000 : 4.49594
Minibatch accuracy: 10.2%
Validation accuracy: 10.3%
Minibatch loss at step 9000 : 3.82125
Minibatch accuracy: 14.8%
Validation accuracy: 16.3%
Minibatch loss at step 10000 : 3.39486
Minibatch accuracy: 17.2%
Validation accuracy: 1

In [16]:
# Creates a graph.
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print sess.run(c)


[[ 22.  28.]
 [ 49.  64.]]
